In [22]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
seed = 18

merged_aspirational_df = pd.read_csv("../../../../data/processed_for_model/merged_themes_using_jaccard_method/merged_Aspirational_sentence_level_batch_1_jaccard.csv", encoding='utf-8')

# Shuffle the merged dataset
merged_aspirational_df = shuffle(merged_aspirational_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_aspirational_df, test_size=0.2, random_state=18, stratify=merged_aspirational_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [23]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (3763, 3) 
Test dataset shape: (941, 3)
Positive labels present in the dataset : 470  out of 3763 or 12.490034546904067%
Positive labels present in the test dataset : 118  out of 941 or 12.539851222104145%


In [24]:
training_df

,sentence,label,phrase
0,this year has been extremely long and challeng...,0,"['So, im here to complete school and walk acro..."
1,i am here at sfsu to gain the knowledge and ex...,1,['I am here at SFSU to gain the knowledge and ...
2,i am taking this sci 115 course so it can help...,0,['Chem 115 is a requirement for me because I a...
3,i have never left the country too which i hate...,0,['I am here because the only thing i want fo b...
4,"with that in mind, i am taking this course so ...",0,['planning on declaring a concentration in zoo...
...,...,...,...
3758,i want to be able to help my family out with p...,0,['I want to get a job in the film industry. Im...
3759,i hope to graduate with my degree and run for ...,0,['I am here because I want to be as educated a...
3760,i am here to learn about physics that are rela...,1,['I am here to learn about physics that are re...
3761,im here to be a better student and a better ve...,1,"[""I'm here to be a better student and a better..."


In [25]:
MAXLEN = 150

In [26]:
X = training_df['sentence']
y = training_df['label']

In [27]:
import ktrain
from ktrain import text

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

model_name = 'bert-base-cased'

distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()


preprocessing train...
language: en
train sequence lengths:
	mean : 20
	95percentile : 39
	99percentile : 55


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 39
	99percentile : 54


In [28]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = {0:0.5713635,1:4.00319149}

[0.5713635  4.00319149]


In [29]:
import tensorflow as tf
import numpy as np
import os
import random
def reset_random_seeds(seed=2):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [30]:
# Build BERT model
# model = text.text_classifier('distilbert', train_data=(X_train, y_train), preproc=distillbert_transformer)
learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)
# learner.fit_onecycle(2e-5, 4, class_weight=class_weights)
# learner.autofit(2.27E-06, early_stopping=4)
learner.set_weight_decay(0.0001)
learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)

# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)

reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 2.27e-06...
Epoch 1/1024


/Users/gbaldonado/miniforge3/envs/ml_env/lib/python3.8/site-packages/ktrain/core.py:415: UserWarning: recompiling model to use AdamWeightDecay as opimizer with weight decay of 0.0001
  warnings.warn(


502/502 [==============================] - 265s 462ms/step - loss: 0.6835 - accuracy: 0.3711 - val_loss: 0.7040 - val_accuracy: 0.5086
Epoch 2/1024
502/502 [==============================] - 211s 420ms/step - loss: 0.6105 - accuracy: 0.6545 - val_loss: 0.5932 - val_accuracy: 0.7025
Epoch 3/1024
502/502 [==============================] - 227s 451ms/step - loss: 0.5202 - accuracy: 0.7219 - val_loss: 0.5603 - val_accuracy: 0.6999
Epoch 4/1024
502/502 [==============================] - 297s 588ms/step - loss: 0.4617 - accuracy: 0.7542 - val_loss: 0.5053 - val_accuracy: 0.7437
Epoch 5/1024
502/502 [==============================] - 391s 778ms/step - loss: 0.4149 - accuracy: 0.7834 - val_loss: 0.5052 - val_accuracy: 0.7437
Epoch 6/1024
502/502 [==============================] - 393s 781ms/step - loss: 0.3798 - accuracy: 0.8013 - val_loss: 0.4634 - val_accuracy: 0.7782
Epoch 7/1024
502/502 [==============================] - 286s 564ms/step - loss: 0.3428 - accuracy: 0.8349 - val_loss: 0.4772 

In [31]:
learner.validate(class_names=distillbert_transformer.get_classes())

24/24 [==============================] - 36s 989ms/step
              precision    recall  f1-score   support

           0       0.96      0.78      0.86       659
           1       0.34      0.80      0.47        94

    accuracy                           0.78       753
   macro avg       0.65      0.79      0.67       753
weighted avg       0.89      0.78      0.81       753



array([[511, 148],
       [ 19,  75]])

In [33]:
distillbert_predictor = ktrain.get_predictor(learner.model, preproc=distillbert_transformer)

distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

y_pred_distillbert = [int(x) for x in y_pred_distillbert]

tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

True Negative: 611, False Positive: 212, False Negative: 24, True Positive: 94
  Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.74      0.84       823
           1       0.31      0.80      0.44       118

    accuracy                           0.75       941
   macro avg       0.63      0.77      0.64       941
weighted avg       0.88      0.75      0.79       941
 



: 

True Negative: 681, False Positive: 142, False Negative: 40, True Positive: 78
  Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.83      0.88       823
           1       0.35      0.66      0.46       118

    accuracy                           0.81       941
   macro avg       0.65      0.74      0.67       941
weighted avg       0.87      0.81      0.83       941

In [32]:
distillbert_predictor.save('../../../../saved_models/aspirational_bert_base_cased_model_08012024_v1') # 0.67 

In [5]:
learner.validate(class_names=distillbert_transformer.get_classes())

51/51 [==============================] - 29s 427ms/step
              precision    recall  f1-score   support

    negative       0.98      0.95      0.97      1440
    positive       0.71      0.88      0.78       185

    accuracy                           0.94      1625
   macro avg       0.85      0.91      0.88      1625
weighted avg       0.95      0.94      0.95      1625



array([[1373,   67],
       [  23,  162]])